Global setup

In [1]:
n=4
k=2
q=41 #Alkaline D
#q should be divisible by 8 for NTT
eta=1 #secret key range, prefered 2 in Alkaline
gamma1=2^4 #y coeff range, power of 2, exponent about 2/3 of q?  Earlier version was *not* power of 2
R.<x> = ZZ[]

In [2]:
q

41

In [3]:
is_prime(q)

True

In [4]:
qlen = ceil(log(q,2))
qlen

6

In [5]:
tau = 1 #number of 1's in c
ln(binomial(4, tau)).n()+tau #challenge entropy, should be between n/2 and n

2.38629436111989

In [6]:
beta = tau*eta
beta

1

Chance of $\mathbf{y}_i$-coefficient passing:

In [7]:
((2*(gamma1-beta)-1)/(2*gamma1-1))^(n*k).n()

0.586530272919658

Expected number of repetitions:

In [8]:
1/(((2*(gamma1-beta)-1)/(2*gamma1-1))^(n*k))^2.n()

2.90682666192675

Alice generates a signing key

In [9]:
A = matrix([[randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q), randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q)], 
            [randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q), randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q)]])
A

[28*x^3 + 39*x^2 + 25*x + 34  20*x^3 + 4*x^2 + 18*x + 18]
[ 35*x^3 + 16*x^2 + 2*x + 31         26*x^3 + 35*x^2 + 8]

In [10]:
s1 = matrix([[randrange(-eta,eta)*x^3+randrange(-eta,eta)*x^2+randrange(-eta,eta)*x+randrange(-eta,eta)], 
            [randrange(-eta,eta)*x^3+randrange(-eta,eta)*x^2+randrange(-eta,eta)*x+randrange(-eta,eta)]])
s1

[    -x^3 - 1]
[-x^2 - x - 1]

In [11]:
s2 = matrix([[randrange(-eta,eta)*x^3+randrange(-eta,eta)*x^2+randrange(-eta,eta)*x+randrange(-eta,eta)], 
            [randrange(-eta,eta)*x^3+randrange(-eta,eta)*x^2+randrange(-eta,eta)*x+randrange(-eta,eta)]])
s2

[-x^3 - x - 1]
[           0]

$s_1$ and $s_2$ are private

In [12]:
t = matrix(((A*s1 + s2) % (x^n+1) % q))
t

[  18*x^3 + 31*x^2 - 3*x - 4]
[37*x^3 - 24*x^2 + 32*x + 24]

$A$ and $t$ are public

Alice encodes her message

In [13]:
M = 'HI_BOB__'
Mlist = [Mod(ord(ch) - ord("A") + 1,q).lift_centered() for ch in M]
Mlist

[8, 9, -10, 2, 15, 2, -10, -10]

Pick nonces and generate $w_1$

In [14]:
y1 = matrix([[randrange(-(gamma1-1), gamma1-1)*x^3+randrange(-(gamma1-1), gamma1-1)*x^2+randrange(-(gamma1-1), gamma1-1)*x+randrange(-(gamma1-1), gamma1-1)], 
            [randrange(-(gamma1-1), gamma1-1)*x^3+randrange(-(gamma1-1), gamma1-1)*x^2+randrange(-(gamma1-1), gamma1-1)*x+randrange(-(gamma1-1), gamma1-1)]])
y1

[-11*x^3 + 3*x^2 - 13*x + 6]
[  9*x^3 - 3*x^2 - 3*x + 11]

In [15]:
y2 = matrix([[randrange(-(gamma1-1), gamma1-1)*x^3+randrange(-(gamma1-1), gamma1-1)*x^2+randrange(-(gamma1-1), gamma1-1)*x+randrange(-(gamma1-1), gamma1-1)], 
            [randrange(-(gamma1-1), gamma1-1)*x^3+randrange(-(gamma1-1), gamma1-1)*x^2+randrange(-(gamma1-1), gamma1-1)*x+randrange(-(gamma1-1), gamma1-1)]])
y2

[-8*x^3 - 14*x^2 + 7*x - 11]
[11*x^3 - 10*x^2 - 10*x - 9]

In [16]:
w1 = matrix(((A*y1+y2) % (x^n+1) % q)).apply_map(lambda f: f.map_coefficients(lambda r: Mod(r,q).lift_centered() ))
w1

[-2*x^3 + 20*x^2 - 18*x + 3]
[13*x^3 + 9*x^2 - 17*x + 16]

Start the "non-cryptographic hash/XOF (combined?)"

In [17]:
w1list = [c for row in w1 for p in row for c in list(p)]  #flatten the matrix down to coefficients
w1list

[3, -18, 20, -2, 16, -17, 9, 13]

In [18]:
def XOF(list1, list2, itemlen, numbits):
    out = ""
    list11 = list1
    while len(out)<numbits:
        newitem = vector(list11).dot_product(vector(list2)) % (2**itemlen) # make sure all bits of input are used
        out = out + format(newitem, f'0{itemlen}b')
        list11 = [newitem] + list11[:-1]
    return out[:numbits]

In [19]:
c_hash = XOF(Mlist, w1list, 6, 20)
c_hash

'01110011110110110010'

Use XOF to sample a sparse polynomial

In [20]:
clenbits = log(n,2)
clenbits

2

In [21]:
def sample_in_ball(digest):  # digest should be at least ****
    c = [0] * n
    k = n
    for i in range(n-tau, n):
        while int(digest[clenbits*k:clenbits*(k+1)], 2) > i:
            k = k+1
        j = int(digest[clenbits*k:clenbits*(k+1)], 2)
        c[i] = c[j]
        c[j] = (-1)^int(digest[i+tau-n], 2)
        k = k+1
    return c

Generate the signature


In [22]:
c = R(sample_in_ball(c_hash))
c

x^3

In [23]:
z1 = matrix((y1 + c*s1) % (x^n+1))
z1

[-12*x^3 + 4*x^2 - 13*x + 6]
[  8*x^3 - 3*x^2 - 2*x + 12]

In [24]:
z2 = matrix((y2 + c*s2) % (x^n+1))
z2

[-9*x^3 - 13*x^2 + 7*x - 10]
[11*x^3 - 10*x^2 - 10*x - 9]

Is $\|\mathbf{z}_1\|_\infty$ or $\|\mathbf{z}_2
\|_\infty \geq \gamma_1-\beta$?

In [25]:
gamma1-beta

15

If so, reject and pick new $\mathbf{y}_i$

If not, $(\mathbf{z}_1, \mathbf{z}_2, c)$ is the signature

In [26]:
(z1, z2, c)

(
[-12*x^3 + 4*x^2 - 13*x + 6]  [-9*x^3 - 13*x^2 + 7*x - 10]     
[  8*x^3 - 3*x^2 - 2*x + 12], [11*x^3 - 10*x^2 - 10*x - 9], x^3
)

Bob verifies the signature

In [27]:
w1prime = matrix((A*z1+z2-c*t) % (x^n+1) % q).apply_map(lambda f: f.map_coefficients(lambda r: Mod(r,q).lift_centered() ))
w1prime

[-2*x^3 + 20*x^2 - 18*x + 3]
[13*x^3 + 9*x^2 - 17*x + 16]

Is $\|\mathbf{z}_i\|_\infty < \gamma_1-\beta$?

In [28]:
z1

[-12*x^3 + 4*x^2 - 13*x + 6]
[  8*x^3 - 3*x^2 - 2*x + 12]

In [29]:
z2

[-9*x^3 - 13*x^2 + 7*x - 10]
[11*x^3 - 10*x^2 - 10*x - 9]

In [30]:
gamma1-beta

15

Yes

Is $c = H(M , w_1')$?

In [31]:
w1primelist = [c for row in w1prime for p in row for c in list(p)]  #flatten the matrix down to coefficients
w1primelist

[3, -18, 20, -2, 16, -17, 9, 13]

In [32]:
cprime_hash = XOF(Mlist, w1primelist, 6, 20)
cprime_hash

'01110011110110110010'

In [33]:
cprime = R(sample_in_ball(cprime_hash))
cprime

x^3

In [34]:
c == cprime

True

Yes.  The signature is accepted.